# Model Training

In [5]:
import torch
import torch.utils.data as Data
import numpy as np
import optuna
from EduKTM import DKT

FILE_PREFIX = 'assistment-2009-2010-skill'

In [ ]:
NUM_QUESTIONS = 123

def get_train_val_loaders(batch_size, val_split=0.2, shuffle=True, data_percentage=1.0):
    # Load the entire dataset
    data = torch.FloatTensor(np.load(FILE_PREFIX + 'train_data.npy'))
    
    # Apply data_percentage to reduce dataset size if needed
    if data_percentage < 1.0:
        total_samples = len(data)
        samples_to_keep = int(total_samples * data_percentage)
        if shuffle:
            indices = torch.randperm(total_samples)[:samples_to_keep]
            data = data[indices]
        else:
            data = data[:samples_to_keep]
    
    # Get dataset size and calculate split
    dataset_size = len(data)
    val_size = int(dataset_size * val_split)
    train_size = dataset_size - val_size
    
    # Split the dataset
    if shuffle:
        indices = torch.randperm(dataset_size)
        train_indices = indices[:train_size]
        val_indices = indices[train_size:]
        train_data = data[train_indices]
        val_data = data[val_indices]
    else:
        train_data = data[:train_size]
        val_data = data[train_size:]
    
    # Create data loaders
    train_loader = Data.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    val_loader = Data.DataLoader(val_data, batch_size=batch_size, shuffle=False)
    
    print(f"Training samples: {train_size}, Validation samples: {val_size}")
    
    return train_loader, val_loader

def get_test_data_loader(batch_size, shuffle=False, data_percentage=1.0):
    data = torch.FloatTensor(np.load(FILE_PREFIX + 'test_data.npy'))
    # Select only a percentage of the data
    if data_percentage < 1.0:
        total_samples = len(data)
        samples_to_keep = int(total_samples * data_percentage)
        if shuffle:
            indices = torch.randperm(total_samples)[:samples_to_keep]
            data = data[indices]
        else:
            data = data[:samples_to_keep]
    
    data_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    return data_loader

In [7]:
def importance_objective(trial):
    # Define hyperparameters to optimize
    hidden_size = trial.suggest_categorical('hidden_size', [5, 10, 20, 50, 100])
    num_layer = trial.suggest_int('num_layers', 1, 3)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    
    train_subset_loader, val_subset_loader = get_train_val_loaders(batch_size, data_percentage=0.05)
    # test_subset_loader = get_test_data_loader( './test_data.npy', batch_size, False, 0.1)
    
    # Initialize and train model
    dkt_model = DKT(NUM_QUESTIONS, hidden_size, num_layer)
    dkt_model.train(train_subset_loader, val_subset_loader, epoch=10, lr=lr)
    
    # Return the AUC score to be maximized
    return dkt_model.eval(val_subset_loader)

importance_study = optuna.create_study(study_name=FILE_PREFIX+"dkt_importances", storage="sqlite:///../studies.db", load_if_exists=True, direction='maximize')
importance_study.optimize(importance_objective, n_trials=100, n_jobs=4)

[I 2025-04-11 19:12:06,876] Using an existing study with name 'assistment-2009-2010-skilldkt_importances' instead of creating a new one.


Training samples: 260, Validation samples: 65
Training samples: 260, Validation samples: 65
Training samples: 260, Validation samples: 65
Training samples: 260, Validation samples: 65


Epoch 0:   0%|          | 0/5 [00:00<?, ?it/s]




Epoch 0:  20%|██        | 1/5 [00:00<00:00,  7.96it/s]


Epoch 0: 100%|██████████| 5/5 [00:00<00:00, 13.81it/s]


Epoch 0: 100%|██████████| 5/5 [00:00<00:00, 13.91it/s]

[Epoch 0] LogisticLoss: 35.642845
[Epoch 0] LogisticLoss: 35.185030



evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0: 100%|██████████| 5/5 [00:00<00:00, 13.48it/s]


...

Epoch 9: 100%|██████████| 17/17 [00:00<00:00, 143.11it/s]


[Epoch 9] LogisticLoss: 6.838573


evaluating: 100%|██████████| 5/5 [00:00<00:00, 312.52it/s]


[Epoch 9] auc: 0.732353


evaluating: 100%|██████████| 5/5 [00:00<00:00, 143.12it/s]
[I 2025-04-11 19:14:00,076] Trial 311 finished with value: 0.7323529011719514 and parameters: {'hidden_size': 10, 'num_layers': 1, 'batch_size': 16, 'learning_rate': 0.012372675178281549}. Best is trial 215 with value: 0.7918122107644532.


In [8]:
print(f"Best parameters: {importance_study.best_params}")
print(f"Best AUC: {importance_study.best_value}")

optuna.visualization.plot_optimization_history(importance_study)
optuna.visualization.plot_param_importances(importance_study)

best_params = importance_study.best_params

Best parameters: {'hidden_size': 20, 'num_layers': 1, 'batch_size': 64, 'learning_rate': 0.02151399408453571}
Best AUC: 0.7918122107644532


In [9]:
optuna.visualization.plot_param_importances(importance_study)

In [10]:
def learning_rate_objective(trial):
    # Use the best parameters from importance_study but optimize learning rate
    hidden_size = 10 # best_params['hidden_size']
    num_layers = 1 # best_params['num_layers']
    batch_size = 64 # best_params['batch_size']
    
    # Define learning rate range to optimize
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    
    # Use smaller subset for faster optimization
    train_subset_loader, val_subset_loader = get_train_val_loaders(batch_size, data_percentage=0.05)
    
    # Initialize model with best parameters from previous study
    dkt_model = DKT(NUM_QUESTIONS, hidden_size, num_layers)
    dkt_model.train(train_subset_loader, val_subset_loader, epoch=10, lr=lr)
    
    # Return validation AUC to be maximized
    return dkt_model.eval(val_subset_loader)

# Create a new study focused on learning rate optimization
lr_study = optuna.create_study(study_name=FILE_PREFIX+ "dkt_learning_rate", storage="sqlite:///../studies.db", load_if_exists=True, direction='maximize')
lr_study.optimize(learning_rate_objective, n_trials=100, n_jobs=4)

[I 2025-04-11 19:14:03,585] Using an existing study with name 'assistment-2009-2010-skilldkt_learning_rate' instead of creating a new one.


Training samples: 260, Validation samples: 65


Epoch 0:  20%|██        | 1/5 [00:00<00:01,  3.67it/s]

Training samples: 260, Validation samples: 65
Training samples: 260, Validation samples: 65


Training samples: 260, Validation samples: 65


...

evaluating: 100%|██████████| 2/2 [00:00<00:00, 168.42it/s]
[I 2025-04-11 19:17:27,677] Trial 298 finished with value: 0.7697030592538592 and parameters: {'learning_rate': 0.055080612482693844}. Best is trial 200 with value: 0.7975189221984604.
Epoch 9: 100%|██████████| 5/5 [00:00<00:00, 20.78it/s]


[Epoch 9] LogisticLoss: 23.248015


evaluating: 100%|██████████| 2/2 [00:00<00:00, 197.32it/s]


[Epoch 9] auc: 0.725049


evaluating: 100%|██████████| 2/2 [00:00<00:00, 198.81it/s]
[I 2025-04-11 19:17:27,830] Trial 299 finished with value: 0.7250491820402858 and parameters: {'learning_rate': 0.03194686122253355}. Best is trial 200 with value: 0.7975189221984604.


In [11]:
# Display results
print(f"Best learning rate: {lr_study.best_params['learning_rate']}")
print(f"Best AUC with optimized learning rate: {lr_study.best_value}")

Best learning rate: 0.03992398922732898
Best AUC with optimized learning rate: 0.7975189221984604


In [12]:
train_loader, val_loader = get_train_val_loaders(batch_size=64)

dkt = DKT(NUM_QUESTIONS, hidden_size=10, num_layers=1)
dkt.train(train_loader, val_loader, epoch=20, lr=lr_study.best_params['learning_rate'])
dkt.save(FILE_PREFIX + "dkt.params")

Training samples: 5211, Validation samples: 1302


Epoch 0: 100%|██████████| 82/82 [00:01<00:00, 61.75it/s]


[Epoch 0] LogisticLoss: 34.150435


evaluating: 100%|██████████| 21/21 [00:00<00:00, 117.56it/s]


[Epoch 0] auc: 0.786268


...

[Epoch 18] auc: 0.794440


Epoch 19: 100%|██████████| 82/82 [00:01<00:00, 59.61it/s]


[Epoch 19] LogisticLoss: 30.361734


evaluating: 100%|██████████| 21/21 [00:00<00:00, 115.13it/s]


[Epoch 19] auc: 0.791506


In [25]:
test_loader = get_test_data_loader(batch_size=64)

dkt.load(FILE_PREFIX + "dkt.params")
auc = dkt.eval(test_loader)
print("auc: %.6f" % auc)

evaluating:   0%|          | 0/286 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 246, got 18